In [1]:
# Baseline bem simples com dados reduzidos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from ast import literal_eval
import os
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'

dataframe = pd.read_csv(DATA_DIR / 'processed_data.csv')

df = dataframe.copy()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1098644 entries, 0 to 1098643
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Unnamed: 0  1098644 non-null  int64 
 1   Id          1098644 non-null  int64 
 2   Body        1098644 non-null  object
 3   Tag         1098644 non-null  object
dtypes: int64(2), object(2)
memory usage: 33.5+ MB


In [3]:
df['Tag'] = df['Tag'].apply(lambda x: literal_eval(x))
df['Tag'] = df['Tag'].apply(lambda x : x[0]) #get first tag which is the most important one
baseline_df = df.sample(frac=0.5) #only fraction of the dataset because of time

In [4]:
# Divisao em train e test
X_train,X_test,y_train,y_test = train_test_split(baseline_df['Body'],baseline_df['Tag'],test_size=0.1,random_state=42)

print(len(X_train),len(X_test))
print(len(y_train),len(y_test))

494389 54933
494389 54933


In [6]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english',max_features=10000)),
    ('classifier', LogisticRegression())
])

pipeline.fit(X_train,y_train)

/home/pedro/Documents/NLP/projeto-NLP/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_features=10000, stop_words='english')),
                ('classifier', LogisticRegression())])

In [7]:
# Testing with X_test

predictions = pipeline.predict(X_test)

accuracy = accuracy_score(y_test,predictions)

print(f"Model accuracy {accuracy}")

Model accuracy 0.5428430997760909


In [16]:
teste = pipeline.predict(["""X = dataset.drop(columns=['target'])
y = dataset['target']

# define categorical and numeric transformers
numeric_transformer = Pipeline(steps=[
    ('knnImputer', KNNImputer(n_neighbors=2, weights="uniform")),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

#  dispatch object columns to the categorical_transformer and remaining columns to numerical_transformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, selector(dtype_exclude="object")),
    ('cat', categorical_transformer, selector(dtype_include="object"))
])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))"""])

teste

array(['python'], dtype=object)

In [17]:
classes = pipeline.named_steps['classifier'].classes_
classes

array(['.htaccess', '.net', 'actionscript-3', 'ajax', 'algorithm',
       'amazon-web-services', 'android', 'android-layout',
       'android-studio', 'angularjs', 'apache', 'api', 'arrays',
       'asp.net', 'asp.net-mvc', 'asp.net-mvc-3', 'asp.net-mvc-4',
       'azure', 'bash', 'batch-file', 'c', 'c#', 'c++', 'c++11', 'class',
       'cocoa', 'cocoa-touch', 'codeigniter', 'cordova', 'css', 'css3',
       'csv', 'database', 'date', 'datetime', 'debugging', 'delphi',
       'django', 'eclipse', 'email', 'entity-framework', 'excel',
       'excel-vba', 'facebook', 'file', 'flash', 'forms', 'function',
       'git', 'google-app-engine', 'google-chrome', 'google-maps',
       'hadoop', 'haskell', 'hibernate', 'html', 'html5', 'http', 'image',
       'internet-explorer', 'ios', 'ipad', 'iphone', 'java', 'javascript',
       'jquery', 'jquery-ui', 'jsf', 'json', 'jsp', 'laravel', 'linq',
       'linux', 'list', 'listview', 'loops', 'magento', 'matlab', 'maven',
       'mongodb', 'ms-access

In [18]:
import pickle

with open('baseline_modelv1.pkl','wb') as f:
    pickle.dump(pipeline,f)  